In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
from static_grader import grader

# Object-oriented exercises
## Introduction

The objective of these exercises is to develop your familiarity with Python's `class` syntax and object-oriented programming. By deepening our understanding of Python objects, we will be better prepared to work with complex data structures and machine learning models. We will develop a `Point` class capable of handling some simple linear algebra operations in 2D.

## Exercise 1: `point_repr`

The first step in defining most classes is to define their `__init__` and `__repr__` methods so that we can construct and represent distinct objects of that class. Our `Point` class should accept two arguments, `x` and `y`, and be represented by a string `'Point(x, y)'` with appropriate values for `x` and `y`.

When you've written a `Point` class capable of this, execute the cell with `grader.score` for this question (do not edit that cell; you only need to modify the `Point` class).

In [ ]:
class Point(object):

    def __init__(self, x , y):
        self.x=x
        self.y=y
    def __repr__(self):
        return "Point(%d, %d)" %(self.x,self.y)


In [ ]:
grader.score.vc__point_repr(lambda points: [str(Point(*point)) for point in points])

Your score:  1.0


## Exercise 2: add_subtract

The most basic vector operations we want our `Point` object to handle are addition and subtraction. For two points $(x_1, y_1) + (x_2, y_2) = (x_1 + x_2, y_1 + y_2)$ and similarly for subtraction. Implement a method within `Point` that allows two `Point` objects to be added together using the `+` operator, and likewise for subtraction. Once this is done, execute the `grader.score` cell for this question (do not edit that cell; you only need to modify the `Point` class.)

(Remember that `__add__` and `__sub__` methods will allow us to use the `+` and `-` operators.)

In [ ]:
from math import sqrt

class Point(object):

    def __init__(self, x, y):
            self.x=x
            self.y=y
    
    def __repr__(self):
        return "Point(%d, %d)" %(self.x,self.y)
    
    def __add__(self, point):
        return Point(self.x+point.x,self.y+point.y)
    def __sub__(self, point):
        return Point(self.x-point.x,self.y-point.y)
    
    def distance(self, point):
        if isinstance(point, Point):
            return sqrt((self.x - point.x)**2+ (self.y-point.y)**2)
        else:
            raise TypeError('Expected point to be a point. Got %s' % type(point))
print(Point(1,3)+Point(3,4))

Point(4, 7)


In [ ]:
from functools import reduce
def add_sub_results(points):
    points = [Point(*point) for point in points]
    return [str(reduce(lambda x, y: x + y, points)), 
            str(reduce(lambda x, y: x - y, points))]

grader.score.vc__add_subtract(add_sub_results)

## Exercise 3: multiplication

Within linear algebra there's many different kinds of multiplication: scalar multiplication, inner product, cross product, and matrix product. We're going to implement scalar multiplication and the inner product.

We can define scalar multiplication given a point $P$ and a scalar $a$ as 
$$aP=a(x,y)=(ax,ay)$$

and we can define the inner product for points $P,Q$ as
$$P\cdot Q=(x_1,y_1)\cdot (x_2, y_2) = x_1x_2 + y_1y_2$$

To test that you've implemented this correctly, compute $2(x, y) \cdot (x, y)$ for a `Point` object. Once this is done, execute the `grader.score` cell for this question (do not edit that cell; you only need to modify the `Point` class.)

(Remember that `__mul__` method will allow us to use the `*` operator. Also don't forget that the ordering of operands matters when implementing these operators.)

In [ ]:
from math import sqrt

class Point(object):

    def __init__(self, x, y):
            self.x=x
            self.y=y
    
    def __repr__(self):
        return "Point(%d, %d)" %(self.x,self.y)
    
    def __mul__(self, point):
        if isinstance(point, int):
            return Point(self.x * point, self.y*point)
        elif isinstance(point, Point):
            return self.x * point.x+ self.y*point.y
        else:
            raise TypeError('Expected point to be int or a point. Got %s' % type(point))
    def distance(self, point):
        if isinstance(point, Point):
            return sqrt((self.x - point.x)**2+ (self.y-point.y)**2)
        else:
            raise TypeError('Expected point to be a point. Got %s' % type(point))
print(Point(1,1)*Point(2,3)*4)

In [ ]:
def mult_result(points):
    points = [Point(*point) for point in points]
    return [point*point*2 for point in points]

grader.score.vc__multiplication(mult_result)

## Exercise 4: Distance

Another quantity we might want to compute is the distance between two points.  This is generally given for points $P_1=(x_1,y_1)$ and $P_2=(x_2,y_2)$ as 
$$D = |P_2 - P_1| = \sqrt{(x_1-x_2)^2 + (y_1-y_2)^2}.$$

Implement a method called `distance` which finds the distance from a point to another point. 

Once this is done, execute the `grader.score` cell for this question (do not edit that cell; you only need to modify the `Point` class.)

### Hint
* *You can use the `sqrt` function from the math package*.

In [ ]:
from math import sqrt

class Point(object):
    
    def __init__(self, x, y):
            self.x=x
            self.y=y
    
    def __repr__(self):
        return "Point(%d, %d)" %(self.x,self.y)
    
    def distance(self, point):
        if isinstance(point, Point):
            return sqrt((self.x - point.x)**2+ (self.y-point.y)**2)
        else:
            raise TypeError('Expected point to be a point. Got %s' % type(point))

In [ ]:
def dist_result(points):
    points = [Point(*point) for point in points]
    return [points[0].distance(point) for point in points]

grader.score.vc__distance(dist_result)

## Exercise 5: Algorithm

Now we will use these points to solve a real world problem!  We can use our Point objects to represent measurements of two different quantities (e.g. a company's stock price and volume).  One thing we might want to do with a data set is to separate the points into groups of similar points.  Here we will implement an iterative algorithm to do this which will be a specific case of the very general $k$-means clustering algorithm.  The algorithm will require us to keep track of two clusters, each of which have a list of points and a center (which is another point, not necessarily one of the points we are clustering).  After making an initial guess at the center of the two clusters, $C_1$ and $C_2$, the steps proceed as follows

1. Assign each point to $C_1$ or $C_2$ based on whether the point is closer to the center of $C_1$ or $C_2$.
2. Recalculate the center of $C_1$ and $C_2$ based on the contained points. 

See [reference](https://en.wikipedia.org/wiki/K-means_clustering#Standard_algorithm) for more information.

This algorithm will terminate in general when the assignments no longer change.  For this question, we would like you to initialize one cluster at `(1, 0)` and the other at `(-1, 0)`.  

The returned values should be the two centers of the clusters ordered by greatest `x` value.  Please return these as a list of numeric tuples $[(x_1, y_1), (x_2, y_2)]$

In order to accomplish this we will create a class called cluster which has two methods besides `__init__` which you will need to write.  The first method `update` will update the center of the Cluster given the points contained in the attribute `points`.  Remember, you after updating the center of the cluster, you will want to reassign the points and thus remove previous assignments.  The other method `add_point` will add a point to the `points` attribute.

Once this is done, execute the `grader.score` cell for this question (do not edit that cell; you only need to modify the `Cluster` class and `compute_result` function.)

In [ ]:
class Cluster(object):
    def __init__(self, x, y):
        self.center = Point(x, y)
        self.points = []
    
    def update(self):
        NewCent=Point(0,0)
        #print(self.points)
        for point in self.points:
            #print(point,NewCent)
            NewCent=NewCent+point
        NewCent.x=float(NewCent.x)/len(self.points)
        NewCent.y=float(NewCent.y)/len(self.points)
        self.center= NewCent
        print("inside update %s" %NewCent)
        self.points = []
        return NewCent
    
    def add_point(self, point):
        self.points.append(point)

In [1]:
def compute_result(points):
    points = [Point(*point) for point in points]
    #print(points[1].distance(Point(0,0)))
    a = Cluster(498,171)
    b = Cluster(-387,-344)
    #a_old = []
    change=False
    while(not change): # max iterations
        a_old=a.center
 
        for point in points:
            if point.distance(a.center) < point.distance(b.center):
                a.add_point(point)
                #if(point in b.points):
                    #b.points.remove(point)
            else:
                b.add_point(point)
                #if(point in a.points):
                    #a.points.remove(point)
        #print(a.points)
        a.update()
        b.update()
        
        if a_old != a.center:
            print("after update %s==%s" %(a.center,a_old))
            break
        #a_old = a.points

    li=[]
    if(a.center.x>b.center.x):
        #print("something %s" %a.center.x)
        li.append((a.center.x,a.center.y))
        li.append((b.center.x,b.center.y))
    else:
        li.append((b.center.x,b.center.y))
        li.append((a.center.x,a.center.y))
    print("finale %s" %li)
    return li

In [ ]:
grader.score.vc__k_means(compute_result)

inside update Point(498, 171)
inside update Point(-387, -344)
after update Point(498, 171)==Point(498, 171)
finale [(498.17391304347825, 171.1304347826087), (-387.25925925925924, -344.6296296296296)]
Your score:  1.0


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*